In [1]:
%run "./ADP_Farmatic_Def"

In [2]:
#################################################################################
""" Process functions for MDM FILES QA DATA GENERATION

"""
 #Who                 When           What
 #Victor Salesa       07/05/2019     Initial Version
################################################################################

####QUALITY DEF##########################################################################################################################
#Pending Unify variables with ADP_Farmatic Sellout

__QUALITY_PFV_DB_TABLE_NAME__='CTL.STG_PROCESS_FILE_VAL' 

__QUALITY_PFV_H_TABLE_NAME__ = 'CTL_PROCESS_FILE_VAL'

__QUALITY_PFV_H_FILE_PATH__  = __QUALITY_BASE_PATH__ + 'ctl_process_file_val'

########################################################################################################################################

def __QA_CANONIZE_MDM_SP_CTL_PROCESS_FILE_DATA__(df):
  try:
    __MDM_ERROR_MESSAGE_JSON__     = """("FILE_NAME_ERROR"="File Name does is not one of {0}_YYYYYMMDDHHMMSS")""".format("_YYYYYMMDDHHMMSS.CSV,".join(__MDM_SP_FILE_PREFIX__)).replace('(','{').replace(')','}')
    __MDM_NOT_ERROR_MESSAGE_JSON__ = """{"FILE_NAME_ERROR":"No Error "}"""
    __MDM_ERROR_CODE_JSON__        = """{"FILE_NAME_ERROR":-1} """
    __MDM_NOT_ERROR_CODE_JSON__    = """{"FILE_NAME_ERROR":0} """
    
    file_db = (df.repartition(32)
        .select(
         col("name").cast(StringType()).alias("FILE_NAME"),
         to_timestamp(col("MASTER_DATA_FILE_TIMESTAMP"), 'yyyyMMddHHmmss').cast(TimestampType()).alias("LANDING_DATE"),
         lit('E').cast(StringType()).alias("VALIDATION_TYPE"),
         lit('MDM').cast(StringType()).alias("PMS_CODE"),
         lit('ES').cast(StringType()).alias("COUNTRY_CODE"),
         lit('').cast(StringType()).alias("PHARMACY_CODE"),
         regexp_replace(col("MASTER_DATA_FILE_NAME"),__MASTER_DATA_FILES_EXT__, "").cast(StringType()).alias("BUSINESS_AREA"),
         col("START_DATE"),
         from_unixtime(unix_timestamp(current_timestamp())).cast(TimestampType()).alias("END_DATE"),
         when(col("IS_MDF_FILE_ERR")==0,lit(0)).otherwise(lit(-1)).cast(DecimalType(18,0)).alias("STATUS"),
         when(col("IS_MDF_FILE_ERR")==0,__MDM_NOT_ERROR_MESSAGE_JSON__).otherwise(__MDM_ERROR_MESSAGE_JSON__).cast(StringType()).alias("MESSAGE_TEXT"),
         when(col("IS_MDF_FILE_ERR")==0,__MDM_NOT_ERROR_CODE_JSON__).otherwise(__MDM_ERROR_CODE_JSON__).cast(StringType()).alias("ERROR_CODE")
     )
     .drop("STATUS_OK")
    )
  except Exception as err:
      ADP_log_exception(process, logger_name, level_action, log_level, "", sys._getframe().f_code.co_name,  sys.exc_info())
      raise Exception(err)
  return file_db
  
#######################################################################################
  
def QA_CANONIZE_MDM_SP_CTL_PROCESS_FILE_DATA(df):
  """Process file data and converts it into canonical format

      Parameters:
      Dataframe df      -- Dataframe with the proper structure
    Return:
      Dataframe -- new dataframe with the CTL.PROCESS_FILE DB TABLE structure

    Example:

  """
  #Who                 When           What
  #Victor Salesa       20/11/2018     Initial version
  #Ana Perez           16/04/2019     Included log managment and exception managment
  
  try:
    return __QA_CANONIZE_MDM_SP_CTL_PROCESS_FILE_DATA__(df)
  except Exception as err:
      ADP_log_exception(process, logger_name, level_action, log_level, "", sys._getframe().f_code.co_name,  sys.exc_info())
      raise Exception(err)

#######################################################################################
  
def QA_SERIALIZE_MDM_SP_FILE_NAME_VALIDATION_DATA(df):
  """Serialize QA data for file name validation
  
      Parameters:
      Dataframe df  -- Dataframe with the proper structure containing file name quality information
    Return:
       Return:
      Dataframe -- new dataframe with the CTL.PROCESS_FILE DB TABLE structure
    Example:
    

  """
  #Who                 When           What
  #Victor Salesa       21/11/2018     Initial version
  #Victor Salesa       11/01/2019     Changed write db output to CTL.PROCESS_FILE_VAL
  #Victor Salesa       11/01/2019     Changed write file output to ctl_process_file_val 
  #Victor Salesa       27/02/2019     Changed write to parquet
  #Victor Salesa       28/02/2019     Added select fields in order and force schema to be sure that schema in file and db is the right one
  #Victor Salesa       18/03/2019     Added STATUS logic to be -1 when the file is not correct.
  #Ana Perez           16/04/2019     Included log managment and exception managment
  #Ana Perez           09/05/2019     Delete print comments
  try:
    df.cache()

    #Transform file name validation process results into canonical
    file_db = df.transform(QA_CANONIZE_MDM_SP_CTL_PROCESS_FILE_DATA).cache()
    file_db = GetDataFrameAsSchema(file_db,__CTL_PROCESS_FILE_VAL_SCHEMA__)
    
    #Write to Folder
    ADP_log_debug(process, logger_name, level_action, log_level, "Before saveAsCanonical ctl_process_file_val: " + str(file_db.count()) + " rows ", sys._getframe().f_code.co_name)
    saveAsCanonical(file_db,__QUALITY_PFV_H_FILE_PATH__,table_name=__QUALITY_PFV_H_TABLE_NAME__,mode='append',debug=True)
    ADP_log_debug(process, logger_name, level_action, log_level, "After saveAsCanonical ctl_process_file_val", sys._getframe().f_code.co_name)
    
    #Write to Database
    ADP_log_debug(process, logger_name, level_action, log_level, "Before saveToDB CTL.PROCESS_FILE_VAL: " + str(file_db.count()) + " rows ", sys._getframe().f_code.co_name)
    saveToDB(file_db,__QUALITY_PFV_DB_TABLE_NAME__,mode="append",debug=True)
    ADP_log_debug(process, logger_name, level_action, log_level, "After saveToDB CTL.PROCESS_FILE_VAL", sys._getframe().f_code.co_name)
    
    return file_db
  except Exception as err:
      ADP_log_exception(process, logger_name, level_action, log_level, "", sys._getframe().f_code.co_name,  sys.exc_info())
      raise Exception(err)
#######################################################################################